In [1]:
%pip install tensorflow
%pip install tensorflow.keras.models
%pip install tensorflow.keras.layers
%pip install scikeras

  Using cached keras-2.15.0-py3-none-any.whl.metadata (2.4 kB)
Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: keras 3.8.0
    Uninstalling keras-3.8.0:
      Successfully uninstalled keras-3.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikeras 0.13.0 requires keras>=3.2.0, but you have keras 2.15.0 which is incompatible.

[notice] A new release of pip is available: 24.1.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement tensorflow.keras.models (from versions: none)
ERROR: No matching distribution found for tensorflow.keras.models

[notice] A new release of pip is available: 24.1.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you m

In [2]:
import matplotlib.pyplot as plt, numpy as np, pandas as pd, seaborn as sb

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.preprocessing import LabelEncoder, StandardScaler

# NN imports
from sklearn.model_selection import GridSearchCV, cross_val_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor

# ensemble 
from sklearn.ensemble import BaggingRegressor

In [3]:
# import datasets and transformations (reusability purposes)
from IPython import get_ipython

get_ipython().run_line_magic('run', 'datasets.ipynb')

/var/folders/31/9bd8ksys1rzbjk8qs9scl95h0000gn/T/ipykernel_83368/549573036.py:4: DtypeWarning: Columns (4,6,20,50,52) have mixed types. Specify dtype option on import or set low_memory=False.
  products_s = pd.read_csv('../datasets/stock_forecasting/2022-2025/[LT] Products [STOCK].txt', sep='\t', header=0) # stock
/var/folders/31/9bd8ksys1rzbjk8qs9scl95h0000gn/T/ipykernel_83368/549573036.py:5: DtypeWarning: Columns (5,48) have mixed types. Specify dtype option on import or set low_memory=False.
  tickets_c_i = pd.read_csv('../datasets/stock_forecasting/2022-2025/[LT] Tickets [CUSTOM] [ITEMS].txt', sep='\t', header=0) # customer order items
/var/folders/31/9bd8ksys1rzbjk8qs9scl95h0000gn/T/ipykernel_83368/549573036.py:6: DtypeWarning: Columns (43,57,105,140,211,246,253,291,295,312,323,326,328,346,350) have mixed types. Specify dtype option on import or set low_memory=False.
  tickets_c_m = pd.read_csv('../datasets/stock_forecasting/2022-2025/[LT] Tickets [CUSTOM] [MAIN].txt', sep='\t', h

-------------- Product [STOCK] ----------------
   Adhesive  Alternate  Available  BackOrdered Box_Size  CaseQty Color  \
0       NaN        NaN          0            0      NaN      NaN   NaN   
1       NaN        NaN          0            0      NaN      NaN   NaN   
2       NaN        NaN          0            0      NaN      NaN   NaN   
3       NaN        NaN          0            0      NaN      NaN   NaN   
4       NaN        NaN          0            0      NaN      NaN   NaN   

   Commission   Cost  Currency_ExchangeRate  ...  SupplierName  SupplierNo  \
0           0   0.00                      0  ...           NaN         NaN   
1           0   0.00                      0  ...           NaN         NaN   
2           0   0.00                      0  ...           NaN         NaN   
3           0   0.00                      0  ...           NaN         NaN   
4           0  49.11                      0  ...           NaN         NaN   

  SupplierNotes SupplierPartNo Tag Tot

/var/folders/31/9bd8ksys1rzbjk8qs9scl95h0000gn/T/ipykernel_83368/3936327507.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  merged_data['OrderDate'] = pd.to_datetime(merged_data['OrderDate'], errors='coerce')
/var/folders/31/9bd8ksys1rzbjk8qs9scl95h0000gn/T/ipykernel_83368/3936327507.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data['OrderDate'] = pd.to_datetime(merged_data['OrderDate'], errors='coerce')
/var/folders/31/9bd8ksys1rzbjk8qs9scl95h0000gn/T/ipykernel_83368/3936327507.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`

In [4]:
# # COULD BE USED FOR FEATURE ENGINEERING

# # Ensure 'OrderDate' and 'Ship_by_Date' are in datetime format
# merged_data['OrderDate'] = pd.to_datetime(merged_data['OrderDate'])
# merged_data['Ship_by_Date'] = pd.to_datetime(merged_data['Ship_by_Date'])

# # 'Lead_Time' in days
# merged_data['Lead_Time'] = (merged_data['Ship_by_Date'] - merged_data['OrderDate']).dt.days
# # missing values in 'Lead_Time'
# merged_data['Lead_Time'].fillna(merged_data['Lead_Time'].median(), inplace=True)

### Evaluation and paramgrids

### Make Model

In [5]:
import sys
sys.path.append('../src')
from model_utils import * # evaluate_model, param grids and hyperparm tuning

# define the features and target variable from 'product_sales'
X = product_sales[['ProductNumber', 'order_month']]
y = product_sales['OrderQuantity']

# convert 'ProductNumber' to numeric values
label_encoder = LabelEncoder()
X['ProductNumber'] = label_encoder.fit_transform(X['ProductNumber'])

# normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# train-test split with scaled features
X_train_scaled, X_val_scaled, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

/var/folders/31/9bd8ksys1rzbjk8qs9scl95h0000gn/T/ipykernel_83368/3097834171.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['ProductNumber'] = label_encoder.fit_transform(X['ProductNumber'])


In [ ]:
# train the model with scaled features (Random Forest Regressor)
rf = RandomForestRegressor()
rf_params = find_best_hyperparameters(rf, param_grids(rf.__class__.__name__), X_train_scaled, y_train)

ValueError: Unsupported model type: RandomForestRegressor()

In [10]:
rf = RandomForestRegressor(**rf_params)
rf.fit(X_train_scaled, y_train)
y_pred_rf = rf.predict(X_val_scaled)

In [11]:
evaluate_model(rf, X_scaled, y)
print('--------------------------------------------')

Evaluating model...
Model: RandomForestRegressor
Mean Absolute Error (MAE): 119274.2412
Mean Squared Error (MSE): 97358800735.8599
Root Mean Squared Error (RMSE): 312023.7182
R-squared (R²): -0.0379
--------------------------------------------


In [12]:
# # train the model with scaled features (Decision Tree Regressor)
# dt = DecisionTreeRegressor()
# dt_params = find_best_hyperparameters(dt, param_grids(dt), X_train_scaled, y_train)

In [13]:
# dt = DecisionTreeRegressor(**dt_params)
# dt.fit(X_train_scaled, y_train)
# y_pred_dt = dt.predict(X_val_scaled)

In [14]:
# evaluate_model(dt, X_scaled, y)
# print('--------------------------------------------')

In [15]:
# # train the model with scaled features (Linear Regression)
# lr = LinearRegression()
# lr_params = find_best_hyperparameters(lr, param_grids(lr), X_train_scaled, y_train)

In [16]:
# lr = LinearRegression(**lr_params)
# lr.fit(X_train_scaled, y_train)
# y_pred_lr = lr.predict(X_val_scaled)

In [17]:
# evaluate_model(lr, X_scaled, y)
# print('--------------------------------------------')

In [18]:
# NN METRICS ARE NOT ARENT BETTER THAN BASE MODELS, SO ITS COMMENTED OUT

# Define the neural network model
def create_nn_model(input_shape):
    model = Sequential()
    model.add(Dense(64, input_dim=input_shape, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
    return model

# Wrap the Keras model with KerasRegressor
input_shape = X_train_scaled.shape[1]
nn_model = KerasRegressor(build_fn=create_nn_model, input_shape=input_shape, verbose=1)

# Perform GridSearchCV
grid_search = GridSearchCV(estimator=nn_model, param_grid=param_grids(nn), cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train_scaled, y_train)

# Get the best parameters
nn_params = grid_search.best_params_
print(f'Best Parameters: {nn_params}')

# Set the best parameters to the model
nn_model.set_params(**nn_params)

# Use cross_val_score or other scikit-learn utilities
scores = cross_val_score(nn_model, X_train_scaled, y_train, cv=5)
print(scores)

# Train the model
history = nn_model.fit(X_train_scaled, y_train, epochs=nn_params['epochs'], batch_size=nn_params['batch_size'], validation_split=0.2, verbose=1)

# Predict using the neural network model
y_pred_nn = nn_model.predict(X_val_scaled)

# Evaluate the model
evaluate_model(nn_model, X_scaled, y)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``buil

Epoch 1/200
Epoch 1/200
Epoch 1/200
Epoch 1/200
Epoch 1/200
312/312 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 97789386752.0000 - mean_absolute_error: 128556.81259   
Epoch 2/200
312/312 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 99071664128.0000 - mean_absolute_error: 128109.33599
Epoch 2/200
312/312 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 119939981312.0000 - mean_absolute_error: 130922.3672
Epoch 2/200
312/312 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 104020877312.0000 - mean_absolute_error: 128993.2344
Epoch 2/200
312/312 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 102231654400.0000 - mean_absolute_error: 129895.5625
Epoch 2/200
312/312 ━━━━━━━━━━━━━━━━━━━━ 0s 900us/step - loss: 97678229504.0000 - mean_absolute_error: 128163.0078
Epoch 3/200
312/312 ━━━━━━━━━━━━━━━━━━━━ 0s 905us/step - loss: 98963070976.0000 - mean_absolute_error: 127721.9688
Epoch 3/200
312/312 ━━━━━━━━━━━━━━━━━━━━ 0s 899us/step - loss: 103910285312.0000 - mean_absolute_error: 128594.3672
Epoch 3/200
312/312 ━━━━━━━━━━━━━

/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
390/390 ━━━━━━━━━━━━━━━━━━━━ 1s 678us/step - loss: 103164370944.0000 - mean_absolute_error: 125927.1328
Epoch 2/200
390/390 ━━━━━━━━━━━━━━━━━━━━ 0s 644us/step - loss: 102953025536.0000 - mean_absolute_error: 125165.0312
Epoch 3/200
390/390 ━━━━━━━━━━━━━━━━━━━━ 0s 659us/step - loss: 102016360448.0000 - mean_absolute_error: 121875.6250
Epoch 4/200
390/390 ━━━━━━━━━━━━━━━━━━━━ 0s 608us/step - loss: 100096753664.0000 - mean_absolute_error: 115916.3906
Epoch 5/200
390/390 ━━━━━━━━━━━━━━━━━━━━ 0s 594us/step - loss: 97348493312.0000 - mean_absolute_error: 109669.8516
Epoch 6/200
390/390 ━━━━━━━━━━━━━━━━━━━━ 0s 657us/step - loss: 94251171840.0000 - mean_absolute_error: 105672.8750
Epoch 7/200
390/390 ━━━━━━━━━━━━━━━━━━━━ 0s 608us/step - loss: 91398676480.0000 - mean_absolute_error: 105396.3359
Epoch 8/200
390/390 ━━━━━━━━━━━━━━━━━━━━ 0s 597us/step - loss: 89233539072.0000 - mean_absolute_error: 108201.4688
Epoch 9/200
390/390 ━━━━━━━━━━━━━━━━━━━━ 0s 593us/step - loss: 87876526080.0

...

/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


312/312 ━━━━━━━━━━━━━━━━━━━━ 1s 594us/step - loss: 97789386752.0000 - mean_absolute_error: 128556.8125
Epoch 2/200
312/312 ━━━━━━━━━━━━━━━━━━━━ 0s 562us/step - loss: 97678229504.0000 - mean_absolute_error: 128163.0078
Epoch 3/200
312/312 ━━━━━━━━━━━━━━━━━━━━ 0s 562us/step - loss: 97152090112.0000 - mean_absolute_error: 126334.2891
Epoch 4/200
312/312 ━━━━━━━━━━━━━━━━━━━━ 0s 563us/step - loss: 95992250368.0000 - mean_absolute_error: 122537.4141
Epoch 5/200
312/312 ━━━━━━━━━━━━━━━━━━━━ 0s 563us/step - loss: 94166163456.0000 - mean_absolute_error: 117464.4297
Epoch 6/200
312/312 ━━━━━━━━━━━━━━━━━━━━ 0s 557us/step - loss: 91823808512.0000 - mean_absolute_error: 112700.3438
Epoch 7/200
312/312 ━━━━━━━━━━━━━━━━━━━━ 0s 566us/step - loss: 89250201600.0000 - mean_absolute_error: 109449.9609
Epoch 8/200
312/312 ━━━━━━━━━━━━━━━━━━━━ 0s 558us/step - loss: 86779584512.0000 - mean_absolute_error: 108352.6328
Epoch 9/200
312/312 ━━━━━━━━━━━━━━━━━━━━ 0s 560us/step - loss: 84689698816.0000 - mean_absol

/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/mwilko777/.pyenv/versions/3.11.0/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


312/312 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 97789386752.0000 - mean_absolute_error: 128556.8125 - val_loss: 117385396224.0000 - val_mean_absolute_error: 129730.0312
Epoch 2/200
312/312 ━━━━━━━━━━━━━━━━━━━━ 0s 724us/step - loss: 97678229504.0000 - mean_absolute_error: 128163.0078 - val_loss: 116999962624.0000 - val_mean_absolute_error: 128373.8125
Epoch 3/200
312/312 ━━━━━━━━━━━━━━━━━━━━ 0s 720us/step - loss: 97152090112.0000 - mean_absolute_error: 126334.2891 - val_loss: 116025647104.0000 - val_mean_absolute_error: 125085.5781
Epoch 4/200
312/312 ━━━━━━━━━━━━━━━━━━━━ 0s 720us/step - loss: 95992250368.0000 - mean_absolute_error: 122537.4141 - val_loss: 114387320832.0000 - val_mean_absolute_error: 120160.0938
Epoch 5/200
312/312 ━━━━━━━━━━━━━━━━━━━━ 0s 723us/step - loss: 94166163456.0000 - mean_absolute_error: 117464.4297 - val_loss: 112191094784.0000 - val_mean_absolute_error: 115070.4844
Epoch 6/200
312/312 ━━━━━━━━━━━━━━━━━━━━ 0s 881us/step - loss: 91823808512.0000 - mean_absolut

KeyboardInterrupt: 

In [ ]:
evaluate_model(nn_model, X_scaled, y)
print('--------------------------------------------')

In [ ]:
# Create subplots
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18, 6))

# Plot the bar plot to compare the actual and predicted values
axes[0].bar(product_sales.loc[y_val.index, 'ProductNumber'], y_val, label='Actual', alpha=0.6)
axes[0].bar(product_sales.loc[y_val.index, 'ProductNumber'], y_pred_rf, label='Predicted', alpha=0.6)
axes[0].set_xlabel('Product Number')
axes[0].set_ylabel('Order Quantity')
axes[0].set_title('Actual vs Predicted Order Quantity')
axes[0].set_ylim(0, 2250000)
axes[0].legend()

# Plot the residual plot
residuals = y_val - y_pred_rf
axes[1].scatter(product_sales.loc[y_val.index, 'ProductNumber'], residuals, alpha=0.6)
axes[1].axhline(y=0, color='r', linestyle='--')
axes[1].set_xlabel('Product Number')
axes[1].set_ylabel('Residuals')
axes[1].set_title('Residual Plot')

# Show the plots
plt.tight_layout()
plt.show()

### Ensemble Methods

In [ ]:
bagging_rf = BaggingRegressor(estimator=RandomForestRegressor(**rf_params), n_estimators=10, random_state=42)
bagging_rf.fit(X_train_scaled, y_train)
y_pred_bagging_rf = bagging_rf.predict(X_val_scaled)

evaluate_model(bagging_rf, X_scaled, y)
print('--------------------------------------------')

In [32]:
# bagging_nn = BaggingRegressor(estimator=KerasRegressor(build_fn=create_nn_model, input_shape=input_shape, **nn_params), n_estimators=10, random_state=42)
# bagging_nn.fit(X_train_scaled, y_train)
# y_pred_bagging_nn = bagging_nn.predict(X_val_scaled)

# evaluate_model(bagging_nn, X_scaled, y)
# print('--------------------------------------------')

NN Bagging: 
Mean Absolute Error (MAE): 145788.9249
Mean Squared Error (MSE): 117073044061.8419
Root Mean Squared Error (RMSE): 342159.3840
R-squared (R²): 0.0013
--------------------------------------------

In [ ]:
from sklearn.ensemble import VotingRegressor

rf_nn_voting = VotingRegressor(estimators=[('rf', RandomForestRegressor(**rf_params)), ('nn', KerasRegressor(build_fn=create_nn_model, input_shape=input_shape, **nn_params))])
rf_nn_voting.fit(X_train_scaled, y_train)
y_pred_rf_nn_voting = rf_nn_voting.predict(X_val_scaled)

evaluate_model(rf_nn_voting, X_scaled, y)
print('--------------------------------------------')

# Voting Ensemble

Mean Absolute Error (MAE): 134711.8210
Mean Squared Error (MSE): 116899234027.1837
Root Mean Squared Error (RMSE): 341905.2998
R-squared (R²): 0.0028
--------------------------------------------

In [ ]:
# Create subplots
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18, 6))

# Plot the bar plot to compare the actual and predicted values
axes[0].bar(product_sales.loc[y_val.index, 'ProductNumber'], y_val, label='Actual', alpha=0.6)
axes[0].bar(product_sales.loc[y_val.index, 'ProductNumber'], y_pred_rf_nn_voting, label='Predicted', alpha=0.6)
axes[0].set_xlabel('Product Number')
axes[0].set_ylabel('Order Quantity')
axes[0].set_title('Actual vs Predicted Order Quantity')
axes[0].set_ylim(0, 2250000)
axes[0].legend()

# Plot the residual plot
residuals = y_val - y_pred_rf_nn_voting
axes[1].scatter(product_sales.loc[y_val.index, 'ProductNumber'], residuals, alpha=0.6)
axes[1].axhline(y=0, color='r', linestyle='--')
axes[1].set_xlabel('Product Number')
axes[1].set_ylabel('Residuals')
axes[1].set_title('Residual Plot')

# Show the plots
plt.tight_layout()
plt.show()